# Table-GPT 
This notebook provides instructions and codes to reproduce the main results in our paper.

## Step 1: Load Results
The experiment results for each model are saved as json files under the ``result`` folder. 

In [1]:
import pandas as pd
result_gpt = pd.read_json("results/result_GPT-3.5.jsonl", lines=True)
result_table_gpt = pd.read_json("results/result_Table-GPT-3.5.jsonl", lines=True)
result_chatgpt = pd.read_json("results/result_ChatGPT.jsonl", lines=True)
result_table_chatgpt = pd.read_json("results/result_Table-ChatGPT.jsonl", lines=True)

In [2]:
display(result_gpt.head(5))

,task,dataset,case,setting,prompt,completion,label
0,Column Finding,Spreadsheets-CF,Excel10013,Zero-Shot,# Task Description: Please look at the table b...,"{""result"": ""BalanceLeftTD""}","{""result"": ""Current Month""}"
1,Column Finding,Spreadsheets-CF,Excel10017,Zero-Shot,# Task Description: Please look at the table b...,"{""result"": ""Day Name""}","{""result"": ""Day Name""}"
2,Column Finding,Spreadsheets-CF,Excel10022,Zero-Shot,# Task Description: Please look at the table b...,"{""result"": ""Percentage SCS_ 6&7""}","{""result"": ""EO""}"
3,Column Finding,Spreadsheets-CF,Excel10026,Zero-Shot,# Task Description: Please look at the table b...,"{""result"": ""WORLD VISION""}","{""result"": ""OXFAM""}"
4,Column Finding,Spreadsheets-CF,Excel10028,Zero-Shot,# Task Description: Please look at the table b...,"{""result"": ""Autism""}","{""result"": ""Intellectual Disabilities""}"


# Step 2: Evaluate Performance
We provide a `Evaluator` class that can compute the performance score for each task. We can use it to evaluate the performance of each model by running the following code.

In [3]:
from evaluator import Evaluator
def evaluate_model(result, model_name):
    # Compute the performance score for each dataset
    zeroshot = []
    fewshot = []
    
    for (task, setting, dataset), res in result.groupby(["task", "setting", "dataset"]):
        evaluator = Evaluator(task=task)
        score = evaluator.compute_score(res)
        eval_result = {
            "task": task,
            "dataset": dataset,
            f"{model_name} {setting}": round(score, 3)
        }
        if setting == "Zero-Shot":
            zeroshot.append(eval_result)
        else:
            fewshot.append(eval_result)
            
    zeroshot = pd.DataFrame(zeroshot).set_index(["task", "dataset"])
    fewshot = pd.DataFrame(fewshot).set_index(["task", "dataset"])
    scores = pd.concat([zeroshot, fewshot], axis=1)
    return scores

scores_gpt = evaluate_model(result_gpt, "GPT-3.5")
scores_table_gpt = evaluate_model(result_table_gpt, "Table-GPT-3.5")
scores_chatgpt = evaluate_model(result_chatgpt, "ChatGPT")
scores_table_chatgpt = evaluate_model(result_table_chatgpt, "Table-ChatGPT")

Here are what the evaluation results look like.

In [4]:
display(scores_table_gpt)

Table-GPT-3.5 Zero-Shot  \
task                         dataset                                            
Column Finding               Spreadsheets-CF                            0.713   
Column Type Annotation       Efthymiou                                  0.886   
                             Limaye                                     0.755   
                             Sherlock                                   0.449   
                             T2D                                        0.875   
Data Imputation              ExcelSynthetic                             0.558   
Entity Matching              Amazon-Google                              0.657   
                             Beer                                       0.727   
                             DBLP-ACM                                   0.847   
                             DBLP-GoogleScholar                         0.861   
                             Fodors-Zagats                              0.872   
                             Walmart-Amazon                             0.691   
                             iTunes-Amazon                              0.788   
Error Detection              Spreadsheets-Real                          0.565   
                             WebTables-Real                             0.643   
Missing Value Identification Column (no seperator)                      0.294   
                             Column (with seperator)                    0.457   
                             Row (no seperator)                         0.851   
                             Row (with seperator)                       0.959   
Row-to-Row Transformation    BingQL-Unit                                  NaN   
                             BingQL-other                                 NaN   
                             FF-GR-Trifacta                               NaN   
                             Headcase                                     NaN   
                             Stackoverflow                                NaN   
Schema Matching              DeepM                                      1.000   
Table Question               WikiTest                                   0.486   

                                                      Table-GPT-3.5 Few-Shot  
task                         dataset                                          
Column Finding               Spreadsheets-CF                           0.817  
Column Type Annotation       Efthymiou                                 0.847  
                             Limaye                                    0.853  
                             Sherlock                                  0.538  
                             T2D                                       0.915  
Data Imputation              ExcelSynthetic                            0.625  
Entity Matching              Amazon-Google                             0.676  
                             Beer                                      0.923  
                             DBLP-ACM                                  0.912  
                             DBLP-GoogleScholar                        0.896  
                             Fodors-Zagats                             0.977  
                             Walmart-Amazon                            0.711  
                             iTunes-Amazon                             0.943  
Error Detection              Spreadsheets-Real                         0.552  
                             WebTables-Real                            0.545  
Missing Value Identification Column (no seperator)                     0.441  
                             Column (with seperator)                   0.643  
                             Row (no seperator)                        0.882  
                             Row (with seperator)                      0.976  
Row-to-Row Transformation    BingQL-Unit                               0.427  
                             BingQL-other      

## Step 3: Compare Evaluation Results
We compare the results of different models (Table 3 in the paper).

In [5]:
summary = pd.concat([scores_gpt, scores_table_gpt, scores_chatgpt, scores_table_chatgpt], axis=1)
summary = summary[["GPT-3.5 Zero-Shot", "Table-GPT-3.5 Zero-Shot", "GPT-3.5 Few-Shot", "Table-GPT-3.5 Few-Shot", "ChatGPT Zero-Shot", "Table-ChatGPT Zero-Shot", "ChatGPT Few-Shot", "Table-ChatGPT Few-Shot"]]
summary = summary.loc[["Column Finding", "Column Type Annotation", "Missing Value Identification", "Table Question", "Data Imputation", "Entity Matching", "Error Detection", "Schema Matching", "Row-to-Row Transformation"]]
display(summary)

GPT-3.5 Zero-Shot  \
task                         dataset                                      
Column Finding               Spreadsheets-CF                      0.461   
Column Type Annotation       Efthymiou                            0.757   
                             Limaye                               0.683   
                             Sherlock                             0.332   
                             T2D                                  0.776   
Missing Value Identification Column (no seperator)                0.261   
                             Column (with seperator)              0.305   
                             Row (no seperator)                   0.768   
                             Row (with seperator)                 0.875   
Table Question               WikiTest                             0.450   
Data Imputation              ExcelSynthetic                       0.423   
Entity Matching              Amazon-Google                        0.153   
                             Beer                                 0.500   
                             DBLP-ACM                             0.402   
                             DBLP-GoogleScholar                   0.206   
                             Fodors-Zagats                        0.083   
                             Walmart-Amazon                       0.268   
                             iTunes-Amazon                        0.000   
Error Detection              Spreadsheets-Real                    0.058   
                             WebTables-Real                       0.077   
Schema Matching              DeepM                                1.000   
Row-to-Row Transformation    BingQL-Unit                            NaN   
                             BingQL-other                           NaN   
                             FF-GR-Trifacta                         NaN   
                             Headcase                               NaN   
                             Stackoverflow                          NaN   

                                                      Table-GPT-3.5 Zero-Shot  \
task                         dataset                                            
Column Finding               Spreadsheets-CF                            0.713   
Column Type Annotation       Efthymiou                                  0.886   
                             Limaye                                     0.755   
                             Sherlock                                   0.449   
                             T2D                                        0.875   
Missing Value Identification Column (no seperator)                      0.294   
                             Column (with seperator)                    0.457   
                             Row (no seperator)                         0.851   
                             Row (with seperator)                       0.959   
Table Question               WikiTest                                   0.486   
Data Imputation              ExcelSynthetic                             0.558   
Entity Matching              Amazon-Google                              0.657   
                             Beer                                       0.727   
                             DBLP-ACM                                   0.847   
                             DBLP-GoogleScholar                         0.861   
                             Fodors-Zagats                              0.872   
                             Walmart-Amazon                             0.691   
                             iTunes-Amazon                              0.788   
Error Detection              Spreadsheets-Real                          0.565   
                             WebTables-Real                             0.643   
Schema Matching              DeepM                                      1.000   
Row-to-Row Transformation    BingQL-Unit                                  NaN   
         

We can compute the average scores over different tasks, which are the results we show in Figure 8 and Figure 9 in the paper.

In [6]:
avg = summary.groupby("task").agg("mean").round(3)
display(avg)

,GPT-3.5 Zero-Shot,Table-GPT-3.5 Zero-Shot,GPT-3.5 Few-Shot,Table-GPT-3.5 Few-Shot,ChatGPT Zero-Shot,Table-ChatGPT Zero-Shot,ChatGPT Few-Shot,Table-ChatGPT Few-Shot
task,,,,,,,,
Column Finding,0.461,0.713,0.683,0.817,0.699,0.807,0.804,0.849
Column Type Annotation,0.637,0.741,0.715,0.788,0.712,0.755,0.753,0.795
Data Imputation,0.423,0.558,0.570,0.625,0.524,0.594,0.609,0.649
Entity Matching,0.230,0.778,0.779,0.863,NaN,0.839,NaN,0.894
Error Detection,0.068,0.604,0.328,0.548,0.068,0.600,0.404,0.618
Missing Value Identification,0.552,0.640,0.648,0.736,0.616,0.662,0.730,0.750
Row-to-Row Transformation,NaN,NaN,0.531,0.651,NaN,NaN,0.635,0.687
Schema Matching,1.000,1.000,1.000,1.000,0.857,1.000,1.000,1.000
Table Question,0.450,0.486,0.455,0.478,0.513,0.521,0.520,0.528
